# Caserta: GCP Test Part 2

This is part two of the test.  Since the data (at this point) is all in GCP (starting with the CSV sitting in the bucket), all operations at this point are done in the Datalab notebook.


### Operations in this notebook.
4.	Move cryptocurrency data from GCS bucket to BigQuery
5.	Create a Google Datalab notebook instance (done prior to the previous step for having all operations recorded in notebooks).
6.	Execute these five queries inside the notebook:
7.	How many coins have a USD price greater than 8000 USD?
8.	What is the total market cap of the top 100 cryptocurrencies (in USD)?
9.	Which coins have an available supply less than 5M USD?
10.	Which 5 coins have seen the greatest percentage growth in the last week?
11.	How many ticker symbols contain the letter "X" ?

**Then**
12.	Download the notebook as a .ipynb file by choosing correct option under "Notebook" on the top left
13.	Email deliverables back to recruiter


### Deliverables:
1.	Google DataLab notebook (.ipynb) containing data ingestion script and SQL queries including answers to above questions
2.	Public URL of Google Cloud Storage bucket containing CoinMarketCap data


### Extra Credits
1.	Explain thinking at each step
2.	Include Markdown text
3.	Log steps and errors
4.	Push to Github Repository
5.	Publicly accessible link to Google Storage bucket
6.	Use Google Cloud APIs instead of web console whenever possible

As before, we set up logging.

In [1]:
import logging
import sys

FORMAT = '%(asctime)-15s %(levelname)-6s %(message)s'
DATE_FORMAT = '%b %d %H:%M:%S'
formatter = logging.Formatter(fmt=FORMAT, datefmt=DATE_FORMAT)
logger = logging.getLogger(__name__)
# Clear the handlers out
for h in reversed(logger.handlers):
    logger.removeHandler(h)
# Add the one handler that is desired.
handler = logging.StreamHandler()
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

Define a function to wait for the database load job.  BigQuery is a bit touchy on loading data.
For example, the errors aren't very good for incorrect schemas.  So it is best to write an explicit ```wait_for_job``` function to output errors during
development.

In [2]:
import time

def wait_for_job(job):
    while True:
        job.reload()
        if job.state == 'DONE':
            if job.error_result:
                raise RuntimeError(job.errors)
            return
        time.sleep(1)

## Importing CSV to BigQuery

This is the code that copies the CSV file in the bucket [https://storage.googleapis.com/jrnorton_caserta_test_cryptocurrencies/cryptocurrency_prices.csv](https://storage.googleapis.com/jrnorton_caserta_test_cryptocurrencies/cryptocurrency_prices.csv) @ gs://jrnorton_caserta_test_cryptocurrencies/cryptocurrency_prices.csv into BigQuery.

All fields are copied as Strings.  It would have been nice to use the proper data types, but BigQuery import errors out for any NULL field that is not being reading into a ```STRING``` or a ```BYTE```.  Therefore all fields are defined to be strings and will be safe casted as needed during the queries.

In [3]:
from google.cloud import bigquery
import os

try:

    logging.info('Creating BigQuery client')
    client = bigquery.Client()
    dataset_id = 'jrnorton_caserta_bq_cybercurrencies'

    # Create a DatasetReference using a chosen dataset ID.
    # The project defaults to the Client's project if not specified.
    # Raises google.api_core.exceptions.AlreadyExists if the Dataset already
    # exists within the project.
    dataset_ref = client.dataset(dataset_id)

    # Construct a full Dataset object to send to the API.
    dataset = bigquery.Dataset(dataset_ref)
    # Specify the geographic location where the dataset should reside.
    dataset.location = 'US'

    logging.info('Creating BigQuery dataset.')
    dataset = client.create_dataset(dataset)  # API request

    logging.info('Creating the Job Configuration including the schema.')
    job_config = bigquery.LoadJobConfig()
    # The schema has to be done in all strings due to BigQuery limitations on NULL
    # fields.  The commented out code are the desired data types which can be
    # safe cast as needed.
    job_config.schema = [
        bigquery.SchemaField('circulating_supply', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('id', 'STRING'), #'INTEGER'),
        bigquery.SchemaField('last_updated', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('market_cap', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('max_supply', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('name', 'STRING'), #'STRING'),
        bigquery.SchemaField('percent_change_1h', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('percent_change_24h', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('percent_change_7d', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('price', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('rank', 'STRING'), #'INTEGER'),
        bigquery.SchemaField('symbol', 'STRING'), #'STRING'),
        bigquery.SchemaField('total_supply', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('volume_24h', 'STRING'), #'FLOAT64'),
        bigquery.SchemaField('website_slug', 'STRING') #'STRING')
    ]

    job_config.skip_leading_rows = 1

    logging.info('Creating the table')
    table_ref = dataset_ref.table('cryptocurrencies')
    table = bigquery.Table(table_ref, schema=job_config.schema)
    table = client.create_table(table)  # API request

    logging.info('Configure loading table from URI')
    uri = 'gs://jrnorton_caserta_test_cryptocurrencies/cryptocurrency_prices.csv'
    load_job = client.load_table_from_uri(
      uri,
      dataset_ref.table('cryptocurrencies')
    )

    assert load_job.job_type == 'load'

    logging.info('Execute load job')
    wait_for_job(load_job)  # Waits for table load to complete.
    logging.info('Successfully loaded the table!')
except Exception as e:
    logging.fatal(str(e))

## How many coins have a USD price greater than 8000 USD?

We will be making this query against the ```price``` column with a safe cast.  Safe cast is of the form ```SAFE_CAST(x AS FLOAT64)```

In [4]:
query_job = client.query("""
    SELECT COUNT(*)
    FROM `jrnorton_caserta_bq_cybercurrencies.cryptocurrencies`
    WHERE SAFE_CAST(price AS FLOAT64) IS NOT NULL AND SAFE_CAST(price AS FLOAT64) > 8000.0
    LIMIT 10""")

try:
    logging.info('Coins > 8000 USD Query')
    results = query_job.result()  # Waits for job to complete.
    for r in results:
        print('{} cryptocurrencies with value greater than 8000 USD'.format(r[0]))
except Exception as e:
    logging.fatal(str(e))

4 cryptocurrencies with value greater than 8000 USD


## What is the total market cap of the top 100 cryptocurrencies (in USD)?

This uses the ```market_cap``` field.

In [5]:
query_job = client.query("""
    SELECT SAFE_CAST(market_cap AS FLOAT64)
    FROM `jrnorton_caserta_bq_cybercurrencies.cryptocurrencies`
    WHERE SAFE_CAST(market_cap AS FLOAT64) IS NOT NULL
    ORDER BY SAFE_CAST(market_cap AS FLOAT64) DESC
    LIMIT 100""")

try:
    logging.info('Total Market Cap Query')
    results = query_job.result()  # Waits for job to complete.
    sum = 0
    for r in results:
        sum += r[0]
    print('Sum of top 100 (market cap) is {} USD'.format(sum))
except Exception as e:
    logging.fatal(str(e))

Sum of top 100 (market cap) is 2.64225799694e+11 USD


## Which coins have an available supply less than 5M USD?

Uses the ```circulating_supply``` field.  The assumption is that availability means that which is in circulation and is available for purchase.

In [6]:
query_job = client.query("""
    SELECT name, SAFE_CAST(circulating_supply AS FLOAT64)
    FROM `jrnorton_caserta_bq_cybercurrencies.cryptocurrencies`
    WHERE SAFE_CAST(circulating_supply AS FLOAT64) IS NOT NULL AND SAFE_CAST(circulating_supply AS FLOAT64) < 5000000
    ORDER BY SAFE_CAST(circulating_supply AS FLOAT64) ASC""")

try:
    logging.info('Coins < 5M available Query')
    results = query_job.result()  # Waits for job to complete.
    print('{:24} : {:12}'.format('NAME', 'CIRCULATING SUPPLY'))
    for r in results:
        name = r[0].decode('utf-8')
        supply = r[1]
        print('{:24} : {:10.2f} USD'.format(name, supply))
except Exception as e:
    logging.fatal(str(e))

NAME                     : CIRCULATING SUPPLY
Project-X                :       0.00 USD
Bit20                    :       1.00 USD
42-coin                  :      42.00 USD
bitBTC                   :      52.00 USD
bitGold                  :     189.00 USD
300 Token                :     300.00 USD
Primalbase Token         :    1250.00 USD
Russian Miner Coin       :    1507.00 USD
IDEX Membership          :    2000.00 USD
KingN Coin               :    2009.00 USD
Sovereign Hero           :    4773.00 USD
bitSilver                :   21761.00 USD
iTicoin                  :   32000.00 USD
BitBar                   :   37920.00 USD
Digix Gold Token         :   42373.00 USD
GeyserCoin               :   46864.00 USD
Bitgem                   :   61178.00 USD
Speed Mining Service     :  104680.00 USD
bitEUR                   :  107706.00 USD
Bitcoin Plus             :  111577.00 USD
Triangles                :  114974.00 USD
Altcoin                  :  123175.00 USD
Wild Beast Block         :  18

## Which 5 coins have seen the greatest percentage growth in the last week?

In this case, the field of interest is ```percent_change_7d```.

In [7]:
query_job = client.query("""
    SELECT name, SAFE_CAST(percent_change_7d AS FLOAT64)
    FROM `jrnorton_caserta_bq_cybercurrencies.cryptocurrencies`
    WHERE SAFE_CAST(percent_change_7d AS FLOAT64) IS NOT NULL
    ORDER BY SAFE_CAST(percent_change_7d AS FLOAT64) DESC
    LIMIT 5
    """)

try:
    logging.info('5 Coins with greatest growth Query')
    results = query_job.result()  # Waits for job to complete.
    print('{:24} : {:10}'.format('NAME', '% Growth/7 days'))
    for r in results:
        name = r[0].decode('utf-8')
        growth = r[1]
        print('{:24} : {:4.2f} %'.format(name, growth))
except Exception as e:
    logging.fatal(str(e))

NAME                     : % Growth/7 days
GCN Coin                 : 405.09 %
Sakura Bloom             : 282.70 %
Accelerator Network      : 179.26 %
Artex Coin               : 154.89 %
GoChain                  : 152.21 %


## How many ticker symbols contain the letter "X" ?

The tickers correspond to the ```symbol``` column.

In [8]:
query_job = client.query("""
    SELECT COUNT(*)
    FROM `jrnorton_caserta_bq_cybercurrencies.cryptocurrencies`
    WHERE symbol LIKE '%X%'""")

try:
    logging.info('Coins with X in ticker symbol Query')
    results = query_job.result()  # Waits for job to complete.
    for r in results:
        print('{} cryptocurrencies with X in the ticker'.format(r[0]))
except Exception as e:
    logging.fatal(str(e))

198 cryptocurrencies with X in the ticker
